In [11]:
import os, sys, io
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

import shutil
from tkinter.filedialog import askdirectory
from tkinter.messagebox import askyesno
from crop.autocrop import crop
import cv2

In [3]:
def readQR(image : cv2.Mat):
    qrCodeDetector = cv2.QRCodeDetector()
    decoded_text, points, _ = qrCodeDetector.detectAndDecode(image)
    if points is not None:
        return decoded_text
    return ''

In [16]:
# find sd path
sd_path = askdirectory(message='Select Path To SD')
assert sd_path
save_dir = askdirectory(message='Select Directory To Save To')
assert save_dir

# confirmation
assert askyesno(message=f'Confirm Image Transfer:\n{sd_path}\n\u2192\n{save_dir}')

# move imgs from sd to pc
save_paths = []
dir = os.fsencode(sd_path)
for file in os.listdir(dir):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):
        # save original image to computer
        src_path = os.path.join(sd_path, filename)
        dest_path = os.path.join(save_dir, filename)
        shutil.move(src_path, dest_path)
        save_paths.append(dest_path)

# create output csv
csv = 'Image Path, Location, Number of Beetles\n'
for path in save_paths:
    image = cv2.imread(path)
    
    # read QR
    trap_label = readQR(image)

    # crop
    warped_rgb = crop(image)

    # run model
    model_path = ''
    model = torch.hub.load(model_path)
    result = model(path)

    # update csv
    csv += f'{path},{trap_label},{result}\n'

csv_path = os.path.join(save_dir, 'output.csv')
with open(csv_path, 'a') as csvfile:
    csvfile.write(csv)

2023-04-07 15:20:45.255 python[85027:7534542] +[CATransaction synchronize] called within transaction
2023-04-07 15:20:45.259 python[85027:7534542] +[CATransaction synchronize] called within transaction
2023-04-07 15:20:46.328 python[85027:7534542] +[CATransaction synchronize] called within transaction


AssertionError: 

: 

In [18]:
csv_title = 'Image Path, Location, Number of Beetles\n'
csv = 'test3\n'
csv_path = os.path.join('.', 'output.csv')
with open(csv_path, 'a+') as csvfile:
    csvfile.seek(0)
    if csvfile.readline().strip() == csv_title.strip():
        csvfile.seek(0, io.SEEK_END)
    else:
        csv = csv_title + csv
    csvfile.write(csv)